Lecture des fichiers j-shis 
Se référer au file format specification sur le site de j-shis (http://www.j-shis.bosai.go.jp/map/JSHIS2/data/DOC/DataFileRule/A-RULES_en.pdf)
Lecture dans l'ordre de :
- Parameters for seismic activity evaluation and Fault shape of EarthQuake’s Traces Hardly Recognized (EQTHR) from surface evidences

In [1]:
#Import des packages
#packages basiques
import re
import pandas as pd
#packages persos
from earthquake import SegmentEarthquake, MultiSegmentEarthquake, PointsEarthquake, MultiPointsEarthquake,\
    DomainEarthquake, ZoneEarthquake
from shape import PlaneShape, MultiPlaneShape, PointShape, MultiPointShape
from magnitude import GutenbergRichter, CharacteristicMag, DiscreteMagnitude
from occurrence import PoissonProcess, BrownianPTProcess, YearFreqProcess

###### Parameters for seismic activity evaluation and Fault shape of EarthQuake’s Traces Hardly Recognized (EQTHR) from surface evidences (P-Y2017-PRM_MAX_LND_A98F_EQTHR_EN.csv)

Fichier en blocs décrivant chacun une source de séisme

Une première ligne décrivant la loi de gutemberg richter doublement tronquée

| Column | Explanation               |
|-----------------------------------------|
| 01 | Fault code                        |
| 02 | Mean recurrence interval (Years) |
| 03 | The minimum magnitude          |
| 04 | The maximum magnitude          |
| 05 | b-value                        |
| 06 | Number of fault planes           |
| 07 | Fault name                        |

Puis une série de lignes décrivant chacune un des rectangles composant la surface de rupture

| Column | Explanation                                                                          |
|--------|--------------------------------------------------------------------------------------|
| 01     | Rectangular fault number                                                             |
| 02     | Longitude of the reference point of rectangular fault (Tokyo datum)                  |
| 03     | Latitude of the reference point of rectangular fault (Tokyo datum)                   |
| 04     | Longitude of the reference point of rectangular fault (Japanese Geodetic Datum 2000) |
| 05     | Latitude of the reference point of rectangular fault (Japanese Geodetic Datum 2000)  |
| 06     | Depth of the upper edge of rectangular fault (km)                                    |
| 07     | Length of rectangular fault (km)                                                     |
| 08     | Width of rectangular fault (km)                                                      |
| 09     | Strike angle (degree)                                                                |
| 10     | Dip angle (degree)                                                                   |

Extrait:

| F004501 | 12800   | 6.8    | 7.4     | 0.9    | 2 | Kiso-sanmyaku-seien fault zone (Main part/Northern segment) |    |      |    |
|---------|---------|--------|---------|--------|---|-------------------------------------------------------------|----|------|----|
| 1       | 137.603 | 35.58  | 137.6   | 35.583 | 2 | 14                                                          | 16 | 44.4 | 90 |
| 2       | 137.728 | 35.673 | 137.725 | 35.676 | 2 | 30                                                          | 18 | 20.6 | 40 |
| F004502 | 9000    | 6.3    | 6.3     | 0.9    | 1 | Kiso-sanmyaku-seien fault zone (Main part/Southern segment) |    |      |    |
| 1       | 137.586 | 35.514 | 137.583 | 35.517 | 2 | 9.4                                                         | 9  | 9.3  | 90 |

In [6]:
#Fonction qui regroupe le numéros des lignes qui commencent par une lettre dans une liste

def detect_blocks(data):
    start_lines = []
    for i, line in enumerate(data):
        if re.search(r'[A-Z]', line):
            start_lines += [i]
    return start_lines


#Lecture d'une première ligne de bloc décrivant la loi de magnitude
#Création d'une loi de gutemberg richter doublement tronquée (voir GutenbergRichter) associée au code de la source

def parse_fault_info(line):
    flt_info = line.split(',')
    eq_code = flt_info[0]
    # On ne retient pas la période de retour car une information plus riche sur la loi de survenance de la source est disponible ailleurs
    # avract = float(flt_info[1])
    min_mag = float(flt_info[2])
    max_mag = float(flt_info[3])
    b_val = float(flt_info[4])
    nb_planes = int(flt_info[5])
    name = flt_info[6]
    if max_mag > min_mag:
        mag = GutenbergRichter(min_mag, b_val, max_mag)
    else:
        mag = CharacteristicMag(min_mag)
    return eq_code, mag, nb_planes, name

#Lecture d'une ligne décrivant un rectangle
#Création d'un rectangle (voir PlaneShape)

def parse_plane_info(line):
    pln_info = line.split(',')
    idx = int(pln_info[0])
    lon = float(pln_info[3])
    lat = float(pln_info[4])
    dep = float(pln_info[5])
    leng = float(pln_info[6])
    wid = float(pln_info[7])
    strike = float(pln_info[8])
    dip = float(pln_info[9])
    plane = PlaneShape(idx, lon, lat, dep, leng, wid, strike, dip)
    return plane


#Lecture du fichier
#Création d'un ensemble de plans (voir MultiPlaneShape)

def parse_eqthr(quakes):
    fil = open('hazard_inputs/P-Y2017-PRM_MAX_LND_A98F_EQTHR_EN.csv')
    data = fil.readlines()[5:]
    #Liste des débuts de blocs
    start_lines = detect_blocks(data)
    for i in start_lines:
        #Lecture de la première ligne du bloc décrivant la magnitude
        eq_code, mag, nb_planes, name = parse_fault_info(data[i])
        plane_list = []
        for j in range(nb_planes):
            #Lecture d'une ligne décrivant un plan
            plane = parse_plane_info(data[i + j + 1])
            plane_list += [plane]
        shape = MultiPlaneShape(plane_list)
        quakes[eq_code] = {'code': eq_code, 'mag': mag, 'shape': shape, 'name': name}
    fil.close()
    return quakes

###### Parameters for seismic activity evaluation 

| Earthquake type | File name                         |
|---------------------------------------------------|
| Parameters for seismic activity evaluation of Characteristic earthquakes occurring in major active fault zones (Maximum case) | P-2017-ACT_MAX_LND_A98F_EN.csv|
| Parameters for seismic activity evaluation of subduction-zone earthquakes (Maximum case)| P-2017 -PRM-ACT_MAX_PME_MTTL_EN.csv |
| Parameters for seismic activity evaluation of Earthquakes occurring on active faults other than major active fault zones | P-2017-PRM-ACT_AVR_LND_AGR1_EN.csv |

Ces 3 fichiers se présentent au même format et peuvent être lus comme des dataframes

Ils décrivent la loi de survenance d'une grande partie des sources

Chaque ligne décrit la loi de survenance d'une source

| Column | Explanation                                                                                                                                                                                                                                            |
|--------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| CODE   | Fault code                                                                                                                                                                                                                                             |
| PROC   | Stochastic process (BPT: BPT processes, POI: Poisson process, COM: Combined BPT and POI, BSI: BPT process (Simultaneous occurring model), PSI: Poisson process (Simultaneous occurring model),SIM: Simultaneous occurring model, XXX: None evaluation) |
| AVRACT | Mean recurrence interval (Years)                                                                                                                                                                                                                       |
| NEWACT | The time of the latest event (Years ago: from reference date)                                                                                                                                                                                          |
| ALPHA  | Variance                                                                                                                                                                                                                                               |
| P_T30  | Probability of occurrence in 30 years                                                                                                                                                                                                                  |
| P_T50  | Probability of occurrence in 50 years                                                                                                                                                                                                                  |
| NAME   | Fault name                                                                                                                                                                                                                                             |

In [ ]:
#Lecture des fichiers d'activité
#Création des lois de survenance (voir PoissonProcess et BrownianPTProcess)
def parse_activity(quakes):
    #Lecture des fichiers
    activity = pd.read_csv('hazard_inputs/P-Y2017-PRM-ACT_MAX_LND_A98F_EN.csv', skiprows=8, skipinitialspace=True,
                           index_col=0)
    activity2 = pd.read_csv('hazard_inputs/P-Y2017-PRM-ACT_AVR_LND_AGR1_EN.csv', skiprows=8, skipinitialspace=True,
                            index_col=0)
    activity = activity.append(activity2)
    activity2 = pd.read_csv('hazard_inputs/P-Y2017-PRM-ACT_MAX_PME_MTTL_EN.csv', skiprows=8, skipinitialspace=True,
                            index_col=0)
    activity = activity.append(activity2)
    for eq_code, row in activity.iterrows():
        #Processus de Poisson: un paramètre
        if row['PROC'] in ['POI', 'PSI'] and row['AVRACT'] != '-':
            process = PoissonProcess(row['AVRACT'])
        #Temps de premier passage: 3 paramètres
        elif row['PROC'] in ['BPT', 'BSI', 'COM']:
            process = BrownianPTProcess(row['AVRACT'], row['NEWACT'], row['ALPHA'])
        else:
            process = ''
        dico = {'process': process}
        if eq_code in quakes:
            quakes[eq_code].update(dico)
        else:
            dico.update({'code': eq_code, 'name': row['NAME']})
            quakes[eq_code] = dico
    return quakes


###### MultiActivity File (MultiActivity.csv)

Lecture des sources étant la réunion de plusieurs sources

On trouve le code, le nom de la source et la liste des codes des sources qu'elle réunit

| CODE    | NAME                                                        | SEGMENTS                |
|---------|-------------------------------------------------------------|-------------------------|
| F01220B | Nishiyama fault zone (Oshima-Oki segment/Nishiyama segment) | F012201;F012202         |
| F01220C | Nishiyama fault zone (Nishiyama segment/Kama-toge segment)  | F012202;F012203         |
| F01220A | Nishiyama fault zone (All segment)                          | F012201;F012202;F012203 |

In [ ]:
def parse_multi(quakes):
    df = pd.read_csv('hazard_inputs/MultiActivity.csv', index_col=0)
    for code, row in df.iterrows():
        quakes[code]['segments'] = row['SEGMENTS'].split(';')
        quakes[code]['process'] = 'multi'
    return quakes

In [ ]:

def parse_frequencies(quakes):
    codes1 = ['LND_CGR5_CRUST', 'LND_CIZU_CRUST', 'LND_CJPS_CRUST', 'PSE_CURA_CRUST']
    codes2 = ['PSE_CPCF_INTER', 'PSE_CPCF_INTRA', 'PSE_CPHL_INTER', 'PSE_CPHL_INTRA']
    for code in codes1:
        file_name = 'hazard_inputs/P-Y2017-PRM-ACT_' + code + '_CV_SM.csv'
        df = pd.read_csv(file_name, skiprows=7, skipinitialspace=True)
        l = code.split('_')
        code, typ = l[1], l[2]
        eqtype = {'CRUST': 1, 'INTER': 2, 'INTRA': 3}[typ]
        for _, row in df.iterrows():
            if row['FRQ'] > 0:
                process = YearFreqProcess(row['FRQ'])
                mag = GutenbergRichter(row['MMN'], row['BVL'])
                shape = PointShape(row['# MNO'], row['WLG'], row['WLA'], row['DEP'])  # , row['STR'], row['DIP']
                code3 = code + '_' + str(int(row['ANO'])) + '_' + str(int(row['# MNO']))
                quakes[code3] = {'code': code3, 'name': code3, 'process': process, 'mag': mag, 'shape': shape,
                                 'eqtype': eqtype}
    for code in codes2:
        file_name = 'hazard_inputs/P-Y2017-PRM-ACT_' + code + '_CV_SM.csv'
        df = pd.read_csv(file_name, skiprows=7, skipinitialspace=True)
        l = code.split('_')
        code, typ = l[1] + '_' + l[2], l[2]
        eqtype = {'CRUST': 1, 'INTER': 2, 'INTRA': 3}[typ]
        for _, row in df.iterrows():
            if row['FRQ'] > 0:
                process = YearFreqProcess(row['FRQ'])
                mag = GutenbergRichter(row['MMN'], row['BVL'])
                shape = PointShape(row['# MNO'], row['WLG'], row['WLA'], row['DEP'])  # , row['STR'], row['DIP']
                code3 = code + '_' + str(int(row['ANO'])) + '_' + str(int(row['# MNO']))
                quakes[code3] = {'code': code3, 'name': code3, 'process': process, 'mag': mag, 'shape': shape,
                                 'eqtype': eqtype}
    return quakes